In [26]:
import numpy as np
import pandas as pd
import sympy
import funcs as fs
from grad_desc import *
from newton import *
from nelder import *

from newton import calculate_next_point_classic
import utils as u
from typing_local import *

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
x, y = sympy.symbols('x y')

In [34]:
funcs = [
    x ** 2 + y ** 2,
    2 * (y - x ** 2) ** 2 + (1 - x) ** 2,
    # y + sympy.log(x ** 2 + y ** 2),
]

In [35]:
def gradient_descent_constant_rate_001(func, start_point):
    return gradient_descend(
        fs.lambdify(func),
        jac=fs.jacobi(func),
        start=start_point,
        learning_rate_function=constant_rate(0.001),
        max_iter=1000,
        stop_point_delta=1e-10,
        stop_function_delta=1e-10
    )


def gradient_descent_constant_rate_0001(func, start_point):
    return gradient_descend(
        fs.lambdify(func),
        jac=fs.jacobi(func),
        start=start_point,
        learning_rate_function=constant_rate(0.0001),
        max_iter=1000,
        stop_point_delta=1e-10,
        stop_function_delta=1e-10
    )


def scipy_nelder_mead_(func, start_point):
    return scipy_nelder_mead(
        fs.lambdify(func),
        x0=start_point,
        max_iterations=1000,
    )


def newton_descent_classic_next_point_01(func, start_point):
    return newton_descend(
        fs.lambdify(func),
        calculate_next_point_func=calculate_next_point_classic,
        learning_rate_function=constant_rate(0.01),
        jac=fs.jacobi(func),
        hess=fs.hessian(func),
        start=start_point,
        max_iter=1000,
        stop_point_delta=1e-10,
        stop_function_delta=1e-10
    )


def newton_descent_classic_next_point_03(func, start_point):
    return newton_descend(
        fs.lambdify(func),
        calculate_next_point_func=calculate_next_point_classic,
        learning_rate_function=constant_rate(0.03),
        jac=fs.jacobi(func),
        hess=fs.hessian(func),
        start=start_point,
        max_iter=1000,
        stop_point_delta=1e-10,
        stop_function_delta=1e-10
    )


def newton_descent_linalg_next_point(func, start_point):
    return newton_descend(
        fs.lambdify(func),
        calculate_next_point_func=calculate_next_point_linear_system,
        learning_rate_function=constant_rate(0.0001),
        jac=fs.jacobi(func),
        hess=fs.hessian(func),
        start=start_point,
        max_iter=1000,
        stop_point_delta=1e-10,
        stop_function_delta=1e-10
    )


def scipy_newton_(func, start_point):
    return scipy_newton_cg(
        fs.lambdify(func),
        jac=fs.jacobi(func),
        hessian=fs.hessian(func),
        x0=start_point,
        max_iterations=1000,
    )


methods = [
    gradient_descent_constant_rate_001,
    gradient_descent_constant_rate_0001,
    scipy_nelder_mead_,
    newton_descent_classic_next_point_01,
    newton_descent_linalg_next_point,
    scipy_newton_,
]

In [50]:
def from_sympy_to_plotable_func(f):
    return fs.tupled(np.vectorize(sympy.lambdify([x, y], f, 'numpy')))

In [ ]:
data = []

for f in funcs:
    for start in [(1, 1), (10, 10), (100, 100)]:
        for method in methods:
            record = method(f, start)
            match record:
                case NewtonOptimizationResult(result, iters, stop, success, path) if iters < 10 and f == funcs[0] and False:
                    u.plot_func(from_sympy_to_plotable_func(f), path, label=str(f) + ' ' + method.__name__).show()
                case GradientOptimizationResult(result, iters, stop, success, path) if f == funcs[1]:
                    u.plot_func(from_sympy_to_plotable_func(f), path, label=str(f) + ' ' + method.__name__).show()
                
            data.append({
                'function': str(f),
                'start_point': str(start),
                'method': method.__name__,
                'success': record.success,
                'iterations': record.iterations,
                'result': record.result,
                'stop_reason': record.stop_reason,
                'path_len': len(record.path if hasattr(record, 'path') else record.simplexes),
                'path': record.path if hasattr(record, 'path') else record.simplexes
            })



In [44]:
pd.DataFrame(data)

,function,start_point,method,success,iterations,result,stop_reason,path_len,path
0,x**2 + y**2,"(1, 1)",gradient_descent_constant_rate_001,False,1000,"[0.13506452244668363, 0.13506452244668363]",StopReason.ITERATIONS,1001,"[[1.0, 1.0], [0.998, 0.998], [0.996004, 0.9960..."
1,x**2 + y**2,"(1, 1)",gradient_descent_constant_rate_0001,False,1000,"[0.8187143764431012, 0.8187143764431012]",StopReason.ITERATIONS,1001,"[[1.0, 1.0], [0.9998, 0.9998], [0.99960004, 0...."
2,x**2 + y**2,"(1, 1)",scipy_nelder_mead_,True,54,"[-3.370149904276273e-07, -8.874879479484205e-08]",Optimization terminated successfully.,54,"[[[1.0, 1.0], [1.0499999999999998, 0.95], [0.9..."
3,x**2 + y**2,"(1, 1)",newton_descent_classic_next_point_01,True,987,"[4.919682409786446e-05, 4.919682409786446e-05]",StopReason.FUNCTION_DELTA,988,"[[1.0, 1.0], [0.99, 0.99], [0.9801, 0.9801], [..."
4,x**2 + y**2,"(1, 1)",newton_descent_linalg_next_point,True,2,"[0.0, 0.0]",StopReason.FUNCTION_DELTA,3,"[[1.0, 1.0], [0.0, 0.0], [0.0, 0.0]]"
5,x**2 + y**2,"(1, 1)",scipy_newton_,True,2,"[0.0, 0.0]",Optimization terminated successfully.,0,[]
6,x**2 + y**2,"(10, 10)",gradient_descent_constant_rate_001,False,1000,"[1.3506452244668374, 1.3506452244668374]",StopReason.ITERATIONS,1001,"[[10.0, 10.0], [9.98, 9.98], [9.96004000000000..."
7,x**2 + y**2,"(10, 10)",gradient_descent_constant_rate_0001,False,1000,"[8.187143764430978, 8.187143764430978]",StopReason.ITERATIONS,1001,"[[10.0, 10.0], [9.998, 9.998], [9.9960004, 9.9..."
8,x**2 + y**2,"(10, 10)",scipy_nelder_mead_,True,61,"[-1.5522813425740537e-07, 5.504061552856179e-07]",Optimization terminated successfully.,61,"[[[10.0, 10.0], [10.5, 9.5], [9.75, 9.25]], [[..."
9,x**2 + y**2,"(10, 10)",newton_descent_classic_next_point_01,False,1000,"[0.0004317124741065828, 0.0004317124741065828]",StopReason.ITERATIONS,1001,"[[10.0, 10.0], [9.9, 9.9], [9.801, 9.801], [9...."


In [38]:
pd.DataFrame(data).to_csv('results3.csv')

In [21]:
np.vectorize(sympy.lambdify([x, y], funcs[1], 'numpy'))([1, 2], [2, 3])

array([10, 11])

In [19]:
np.vectorize(fs.lambdify(funcs[0]))([[1, 2], [2, 3]])

TypeError: 'numpy.int64' object is not iterable

In [65]:
u.plot_func(
    from_sympy_to_plotable_func(funcs[1]), [], limit=3
).show(renderer='browser')

In [77]:
ff = sympy.sqrt(x ** 2 + y ** 2)

In [78]:
ff

sqrt(x**2 + y**2)

In [86]:
sympy.lambdify(list(ff.free_symbols), ff)(np.sqrt(16), np.sqrt(9))

5.0